In [1]:
import numpy as np
import PIL.Image, base64
from cStringIO import StringIO
import traitlets, ipywidgets as ipyw

In [2]:
yg,xg = np.mgrid[1:80, 1:120]
x0,y0=60., 40.
sigma = 10.
data = np.exp(-((xg-x0)**2+(yg-y0)**2)/2/sigma**2)
data = (data*255.).astype('uint8')

In [3]:
f = StringIO()
PIL.Image.fromarray(data).save(f, 'png')
imgb64v = base64.b64encode(f.getvalue())

In [4]:
class MyImage(ipyw.DOMWidget):
    _view_name = traitlets.Unicode('MyImageView').tag(sync=True)
    _view_module = traitlets.Unicode('myimage').tag(sync=True)

    _b64value = traitlets.Unicode().tag(sync=True)
    _offsetX = traitlets.Int(0).tag(sync=True)
    _offsetY = traitlets.Int(0).tag(sync=True)
    _pixelValue = traitlets.Float().tag(sync=True)
    
    @traitlets.observe('_offsetX', '_offsetY')
    def offset_change(self, change):
        # print(change['name'], change['new'])
        # print self._offsetX, self._offsetY
        x = int(1.*self._offsetX/360*120)
        y = int(1.*self._offsetY/240*80)
        if x >= data.shape[1]: x = data.shape[1]-1
        if y >= data.shape[0]: y = data.shape[0]-1
        self._pixelValue = float(data[y, x])
        return

In [5]:
# hard-code width height for now
width = 360
height = 240

In [6]:
%%javascript
require.undef('myimage');

define('myimage', ["jupyter-js-widgets"], function(widgets) {

    var MyImageView = widgets.DOMWidgetView.extend({

        render: function() {
            //console.log(this.$el);
            var image_src = 'data:image/' + this.model.get('format') + ';base64,' + this.model.get('_b64value');
            var img = $('<img>');
            img.attr('src', image_src);
            img.attr('width', '360px'); img.attr('height', '240px');
            this.$el.append(img);
            //console.log("ok");
            // elements for displayying offsets
            var offsetx = $('<span>'); offsetx.addClass('img-offsetx'); this.$el.append(offsetx);
            //console.log("ok");
            this.$el.append(',&nbsp;')
            var offsety = $('<span>'); offsety.addClass('img-offsety'); this.$el.append(offsety);
            this.$el.append(':&nbsp;')
            // element for pixel value
            var val = $('<span>'); val.addClass('img-val'); this.$el.append(val);
            // handle mouse move
            var widget = this;
            img.mousemove(function(event){
                // console.log(event);
                widget.model.set('_offsetX', event.offsetX); 
                //console.log(widget.model.get('_offsetX'));
                widget.model.set('_offsetY', event.offsetY);
                //widget.model.save_changes();
                widget.touch();
                offsetx.text(Math.floor(event.offsetX*1./360*120)); 
                offsety.text(Math.floor(event.offsetY*1./240*80));
                //console.log(widget.model.get('_offsetY'));
            });
            
            this.model.on('change:_pixelValue', this.on_pixelvalue_change, this);
        },

        on_pixelvalue_change: function(){
            this.$el.children('.img-val').text(this.model.get('_pixelValue'));
        },
    });

    return {
        MyImageView : MyImageView
    };
});



<IPython.core.display.Javascript object>

In [7]:
w = MyImage()
w._b64value = imgb64v
w